In [1]:
import os
from glob import glob

import rasterio as rst
import rastertools as rtools
from pyogrio import read_dataframe, write_dataframe

wrk_dir = r'D:\.temp'

### Stacking bands from a Sentinel-2 .zip

In [ ]:
rtools.s2stack(
    os.path.join(wrk_dir, 'S2B_MSIL2A_20240604T101559_N0510_R065_T32TNT_20240604T130328.SAFE.zip'),
    out_path=os.path.join(wrk_dir, 's2img_10m.tif'), 
    resolution=10
)
rtools.s2stack(
    os.path.join(wrk_dir, 'S2B_MSIL2A_20240604T101559_N0510_R065_T32TNT_20240604T130328.SAFE.zip'),
    out_path=os.path.join(wrk_dir, 's2img_20m.tif'), 
    resolution=[10, 20]
)

## The Raster class

In [ ]:
with rtools.Raster(os.path.join(wrk_dir, 'inputs', 'raster1.tif')) as src:
    image = src.read()

image.__dict__

### Vectorizing a raster

In [10]:
gdf = rtools.vectorize(os.path.join(wrk_dir, 'binary_vegetation.tif'), nodata=0)
write_dataframe(gdf, os.path.join(wrk_dir, 'binary_vegetation.geojson'))

In [11]:
with rst.open(os.path.join(wrk_dir, 'binary_vegetation.tif')) as src:
    img = src.read(1)
    nodata = src.nodata
    transform = src.transform
    crs = src.crs

gdf = rtools.vectorize(img, transform=transform, crs=crs, nodata=0)
write_dataframe(gdf, os.path.join(wrk_dir, 'binary_vegetation2.geojson'))

### Resampling a raster

In [15]:
img = rtools.resample(os.path.join(wrk_dir, 'rasters', 'raster1.tif'), res=1.2, resampling='bilinear')

with rst.open(os.path.join(wrk_dir, 'raster1_resampled.tif'), 'w', **img.attrs) as dst:
    dst.write(img)

### Merging rasters

In [24]:
filelist = glob(os.path.join(wrk_dir, 'rasters', '*.tif'))
img = rtools.merge(filelist)

with rst.open(os.path.join(wrk_dir, 'rasters_merged.tif'), 'w', **img.attrs) as dst:
    dst.write(img)

### Finding the intersect between shapefiles/rasters

In [2]:
rasters = glob(os.path.join(wrk_dir, 'rasters', '*.tif'))
shps = glob(os.path.join(wrk_dir, 'shps', '*.shp'))

outfiles = rtools.intersect(rasters+shps)

In [21]:
from pathlib import Path
import xarray as xr
import geopandas as gpd

for fp, outfile in zip(rasters+shps, outfiles):
    if isinstance(outfile, xr.DataArray):
        with rst.open(os.path.join(wrk_dir, f'{Path(fp).stem}_intersect.tif'), 'w', **outfile.attrs) as dst:
            dst.write(outfile)
    elif isinstance(outfile, gpd.GeoDataFrame):
        write_dataframe(outfile, os.path.join(wrk_dir, f'{Path(fp).stem}_intersect.geojson'))
    else:
        print('How awkward, you messed up in a live demo.')